In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/Charity Funding Predictor/Resources/


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Charity Funding Predictor/Resources


In [18]:
#  Import and read the charity data.csv
import pandas as pd 
df = pd.read_csv("charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
#Drop 'EIN' and 'Name' and 'SPECIAL_CONSIDERATIONS' columns 
df = df.drop(columns=['EIN', 'NAME','SPECIAL_CONSIDERATIONS', 'ORGANIZATION'])
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,1,0,5000,1
1,T3,Independent,C2000,Preservation,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,1,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,1,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,1,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,1,0,5000,0
34297,T5,Independent,C3000,ProductDev,1,0,5000,1


In [20]:
#Determine the number of unique values for each column
df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [21]:
# Look at APPLICATION_TYPE value counts for binning
binning_count = df['APPLICATION_TYPE'].value_counts()
binning_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(binning_count[binning_count<500].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [23]:
# Look at CLASSIFICATION value counts for binning
classification_binning = df['CLASSIFICATION'].value_counts()
classification_binning

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1820        1
C1283        1
C4500        1
C1245        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [24]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = list(classification_binning[classification_binning < 100].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [25]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(df,dtype=float)
df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1270,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2700,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Split our preprocessed data into our features and target arrays
Y = df["IS_SUCCESSFUL"]
X = df.drop(columns=["IS_SUCCESSFUL"])


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 78)



In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units= 100, input_dim=features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units= 50, activation='sigmoid'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units= 30, activation='sigmoid'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units= 50, activation='sigmoid'))

nn.add(tf.keras.layers.Dense(units= 60, activation='relu'))

nn.add(tf.keras.layers.Dense(units= 40, activation='sigmoid'))

nn.add(tf.keras.layers.Dense(units= 40, activation='sigmoid'))

nn.add(tf.keras.layers.Dense(units= 40, activation='relu'))

nn.add(tf.keras.layers.Dense(units= 40, activation='sigmoid'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 100)               4400      
                                                                 
 dense_6 (Dense)             (None, 50)                5050      
                                                                 
 dense_7 (Dense)             (None, 30)                1530      
                                                                 
 dense_8 (Dense)             (None, 50)                1550      
                                                                 
 dense_9 (Dense)             (None, 60)                3060      
                                                                 
 dense_10 (Dense)            (None, 40)                2440      
                                                                 
 dense_11 (Dense)            (None, 40)               

In [29]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [30]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 3s 3ms/step - loss: 0.6123 - accuracy: 0.6833
Epoch 2/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5773 - accuracy: 0.7281
Epoch 3/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5703 - accuracy: 0.7283
Epoch 4/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5647 - accuracy: 0.7293
Epoch 5/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5627 - accuracy: 0.7285
Epoch 6/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5610 - accuracy: 0.7294
Epoch 7/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5597 - accuracy: 0.7303
Epoch 8/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5585 - accuracy: 0.7309
Epoch 9/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5584 - accuracy: 0.7313
Epoch 10/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5577 - accura

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5625 - accuracy: 0.7236 - 475ms/epoch - 2ms/step
Loss: 0.5624586343765259, Accuracy: 0.7236151695251465


In [32]:
# Export our model to HDF5 file
from google.colab import files

nn.save('/content/AlphabetSoupCharity_Optimzation4.h5')
nn.save('AlphabetSoupCharity_Optimzation4.h5')
